In [2]:
import numpy as np
import os
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import matplotlib.pyplot as plt
from scipy.special import softmax
from collections import OrderedDict

import torch
import torch.nn as nn

import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

from framework.mtl_model import MTLModel
from framework.trainer import Trainer
from data.heads.pixel2pixel import ASPPHeadNode

from data.dataloader.cityscapes_dataloader import CityScapes
from data.metrics.pixel2pixel_loss import CityScapesCriterions
from data.metrics.pixel2pixel_metrics import CityScapesMetrics

from data.dataloader.nyuv2_dataloader import NYU_v2
from data.metrics.pixel2pixel_loss import NYUCriterions
from data.metrics.pixel2pixel_metrics import NYUMetrics

from data.dataloader.taskonomy_dataloader import Taskonomy
from data.metrics.pixel2pixel_loss import TaskonomyCriterions
from data.metrics.pixel2pixel_metrics import TaskonomyMetrics

/opt/anaconda/envs/multiDONN/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/shz15022/AutoMTL/automtl_code/data/dataloader/nyuv2_dataloader.py:48: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if self.task is 'normal':
/home/shz15022/AutoMTL/automtl_code/data/dataloader/nyuv2_dataloader.py:65: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if self.task is 'segment_semantic' or self.task is 'normal':
/home/shz15022/AutoMTL/automtl_code/data/dataloader/nyuv2_dataloader.py:65: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if self.task is 'segment_semantic' or self.task is 'normal':
/home/shz15022/AutoMTL/automtl_code/data/dataloader/nyuv2_dataloader.py:85: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if self.task is 'segment_semantic':
/home/shz15022/AutoMTL

# Data

In [9]:
dataroot = 'datasets/cityscapes/'
tasks = ['segment_semantic', 'depth_zbuffer']
task_cls_num = {'segment_semantic': 19, 'depth_zbuffer': 1}

headsDict = nn.ModuleDict()
trainDataloaderDict = {task: [] for task in tasks}
valDataloaderDict = {}
criterionDict = {}
metricDict = {}

for task in tasks:
    headsDict[task] = ASPPHeadNode(512, task_cls_num[task])

    # For model trainer
    dataset = CityScapes(dataroot, 'train', task, crop_h=224, crop_w=224)
    trainDataloaderDict[task].append(DataLoader(dataset, 16, shuffle=True))
    dataset1 = CityScapes(dataroot, 'train1', task, crop_h=224, crop_w=224)
    trainDataloaderDict[task].append(DataLoader(dataset1, 16, shuffle=True)) # for network param training
    dataset2 = CityScapes(dataroot, 'train2', task, crop_h=224, crop_w=224)
    trainDataloaderDict[task].append(DataLoader(dataset2, 16, shuffle=True)) # for policy param training

    dataset = CityScapes(dataroot, 'test', task)
    valDataloaderDict[task] = DataLoader(dataset, 8, shuffle=True)

    criterionDict[task] = CityScapesCriterions(task)
    metricDict[task] = CityScapesMetrics(task)

In [ ]:
dataroot = 'datasets/NYUv2/'
tasks = ['segment_semantic', 'normal', 'depth_zbuffer']
task_cls_num = {'segment_semantic': 40, 'normal':3, 'depth_zbuffer': 1}

headsDict = nn.ModuleDict()
trainDataloaderDict = {task: [] for task in tasks}
valDataloaderDict = {}
criterionDict = {}
metricDict = {}

for task in tasks:
    headsDict[task] = ASPPHeadNode(512, task_cls_num[task])

    # For model trainer
    dataset = NYU_v2(dataroot, 'train', task, crop_h=321, crop_w=321)
    trainDataloaderDict[task].append(DataLoader(dataset, 16, shuffle=True))
    dataset1 = NYU_v2(dataroot, 'train1', task, crop_h=321, crop_w=321)
    trainDataloaderDict[task].append(DataLoader(dataset1, 16, shuffle=True)) # for network param training
    dataset2 = NYU_v2(dataroot, 'train2', task, crop_h=321, crop_w=321) 
    trainDataloaderDict[task].append(DataLoader(dataset2, 16, shuffle=True)) # for policy param training

    dataset = NYU_v2(dataroot, 'test', task, crop_h=321, crop_w=321)
    valDataloaderDict[task] = DataLoader(dataset, 8, shuffle=True)

    criterionDict[task] = NYUCriterions(task)
    metricDict[task] = NYUMetrics(task)

In [4]:
dataroot = 'datasets/Taskonomy/'
tasks = ['segment_semantic', 'normal', 'depth_zbuffer', 'keypoints2d', 'edge_texture']
task_cls_num = {'segment_semantic': 17, 'normal': 3, 'depth_zbuffer': 1, 'keypoints2d': 1, 'edge_texture': 1}

headsDict = nn.ModuleDict()
trainDataloaderDict = {task: [] for task in tasks}
valDataloaderDict = {}
criterionDict = {}
metricDict = {}

for task in tasks:
    headsDict[task] = ASPPHeadNode(512, task_cls_num[task])

    # For model trainer
    dataset = Taskonomy(dataroot, 'train', task, crop_h=224, crop_w=224)
    trainDataloaderDict[task].append(DataLoader(dataset, batch_size=16, shuffle=True))
    dataset1 = Taskonomy(dataroot, 'train1', task, crop_h=224, crop_w=224)
    trainDataloaderDict[task].append(DataLoader(dataset1, 16, shuffle=True)) # for network param training
    dataset2 = Taskonomy(dataroot, 'train2', task, crop_h=224, crop_w=224) 
    trainDataloaderDict[task].append(DataLoader(dataset2, 16, shuffle=True)) # for policy param training

    dataset = Taskonomy(dataroot, 'test_small', task, crop_h=224, crop_w=224)
    valDataloaderDict[task] = DataLoader(dataset, batch_size=8, shuffle=True)

    criterionDict[task] = TaskonomyCriterions(task, dataroot)
    metricDict[task] = TaskonomyMetrics(task, dataroot)

/home/shz15022/AutoMTL/automtl_code/data/dataloader/taskonomy_dataloader.py:19: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if self.mode is 'train':
/home/shz15022/AutoMTL/automtl_code/data/dataloader/taskonomy_dataloader.py:92: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if self.task is not 'normal':
/home/shz15022/AutoMTL/automtl_code/data/dataloader/taskonomy_dataloader.py:97: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if self.task is 'segment_semantic' or self.task is 'depth_zbuffer' or self.task is 'normal':
/home/shz15022/AutoMTL/automtl_code/data/dataloader/taskonomy_dataloader.py:97: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if self.task is 'segment_semantic' or self.task is 'depth_zbuffer' or self.task is 'normal':
/home/shz15022/AutoMTL/automtl_code/data/dataloader/taskonomy_dataloader.py:97: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if self.task is 'segment_semantic' or self.task is 'depth_zbuffer' or 

FileNotFoundError: [Errno 2] No such file or directory: 'datasets/Taskonomy/semseg_prior_factor.npy'

# Model

In [10]:
prototxt = 'models/deeplab_resnet34_adashare.prototxt'
# prototxt = 'models/mobilenetv2.prototxt' # the input dim of heads should be changed to 1280
# prototxt = 'models/mnasnet.prototxt' # the input dim of heads should be changed to 1280
mtlmodel = MTLModel(prototxt, headsDict)
mtlmodel = mtlmodel.cuda()

In [ ]:
# from pytorch2caffe import pytorch2caffe

# mtlmodel.eval()

# name = "aaa"
# dummy_input=torch.ones([1,3,224,224])
# pytorch2caffe.trans_net(mtlmodel, dummy_input, name)
# pytorch2caffe.save_prototxt('{}.prototxt'.format(name))

: 

In [5]:
mtlmodel

MTLModel(
  (headsDict): ModuleDict(
    (segment_semantic): ASPPHeadNode(
      (fc1): Classification_Module(
        (conv1): Conv2d(512, 1024, kernel_size=(3, 3), stride=(1, 1), padding=(6, 6), dilation=(6, 6))
        (conv2): Conv2d(1024, 1024, kernel_size=(1, 1), stride=(1, 1))
        (conv3): Conv2d(1024, 19, kernel_size=(1, 1), stride=(1, 1))
        (relu): ReLU(inplace=True)
        (dropout): Dropout(p=0.5, inplace=False)
      )
      (fc2): Classification_Module(
        (conv1): Conv2d(512, 1024, kernel_size=(3, 3), stride=(1, 1), padding=(12, 12), dilation=(12, 12))
        (conv2): Conv2d(1024, 1024, kernel_size=(1, 1), stride=(1, 1))
        (conv3): Conv2d(1024, 19, kernel_size=(1, 1), stride=(1, 1))
        (relu): ReLU(inplace=True)
        (dropout): Dropout(p=0.5, inplace=False)
      )
      (fc3): Classification_Module(
        (conv1): Conv2d(512, 1024, kernel_size=(3, 3), stride=(1, 1), padding=(18, 18), dilation=(18, 18))
        (conv2): Conv2d(1024, 1024, 

# Train

In [11]:
checkpoint = 'checkpoint/'
trainer = Trainer(mtlmodel, trainDataloaderDict, valDataloaderDict, criterionDict, metricDict, 
                  print_iters=100, val_iters=500, save_num=1, policy_update_iters=100)

### pre_train

In [ ]:
trainer.pre_train(iters=10000, lr=0.0001, savePath=checkpoint+'Cityscapes/')

### alter_train

In [ ]:
loss_lambda = {'segment_semantic': 1, 'depth_zbuffer': 1, 'policy':0.0005}
trainer.alter_train_with_reg(iters=20000, policy_network_iters=(100,400), 
                             policy_lr=0.01, network_lr=0.0001, 
                             loss_lambda=loss_lambda,
                             savePath=checkpoint+'Cityscapes/')

### sample policy from trained policy distribution and save

In [ ]:
policy_list = {'segment_semantic': [], 'depth_zbuffer': []}
name_list = {'segment_semantic': [], 'depth_zbuffer': []}

In [ ]:
for name, param in mtlmodel.named_parameters():
    if 'policy' in name and not torch.eq(param, torch.tensor([0., 0., 0.]).cuda()).all():
        if 'segment_semantic' in name:
            policy_list['segment_semantic'].append(param.data.cpu().detach().numpy())
            name_list['segment_semantic'].append(name)
        elif 'depth_zbuffer' in name:
            policy_list['depth_zbuffer'].append(param.data.cpu().detach().numpy())
            name_list['depth_zbuffer'].append(name)

In [ ]:
sample_policy_dict = OrderedDict()
for task in tasks:
    for name, policy in zip(name_list[task], policy_list[task]):
        distribution = softmax(policy, axis=-1)
        distribution /= sum(distribution)
        choice = np.random.choice((0,1,2), p=distribution)
        if choice == 0:
            sample_policy_dict[name] = torch.tensor([1.0,0.0,0.0]).cuda()
        elif choice == 1:
            sample_policy_dict[name] = torch.tensor([0.0,1.0,0.0]).cuda()
        elif choice == 2:
            sample_policy_dict[name] = torch.tensor([0.0,0.0,1.0]).cuda()

In [ ]:
sample_state = {'state_dict': sample_policy_dict}
torch.save(sample_state, checkpoint+'Cityscapes/' + 'sample_policy.model')

### post train from scratch

In [ ]:
loss_lambda = {'segment_semantic': 1, 'depth_zbuffer': 1}
trainer.post_train(iters=30000, lr=0.001, 
                   decay_lr_freq=4000, decay_lr_rate=0.5,
                   loss_lambda=loss_lambda,
                   savePath=checkpoint+'Cityscapes/', 
                   reload='sample_policy.model')

### get the validation results in the paper 

In [12]:
mtlmodel.load_state_dict(torch.load('checkpoint/CityScapes.model'))
trainer.validate('mtl', hard=True) 

[Iter 1 Task segm] Val Loss: 0.2796
{'mIoU': 0.4282, 'Pixel Acc': 0.7482, 'cmp': 0.0935}
[Iter 1 Task dept] Val Loss: 0.0173
{'abs_err': 0.0184, 'rel_err': 0.3374, 'sigma_1.25': 70.0025, 'sigma_1.25^2': 86.6344, 'sigma_1.25^3': 93.4961, 'cmp': 0.1647}


: 

# Inference

In [ ]:
x = torch.from_numpy(np.random.random([32, 3, 3, 3])).float()
mtlmodel.load_state_dict(torch.load('CityScapes.model'))
output = mtlmodel(x, task='segment_semantic', hard=True)